In [1]:
# default_exp protein_intensity_estimation

In [2]:
%reload_ext autoreload

%autoreload 2

In [3]:
#export
import pandas as pd
import numpy as np
import directlfq.normalization as lfqnorm

def estimate_protein_intensities(normed_df, min_nonan, maximum_df_length):
    "derives protein pseudointensities from between-sample normalized data"
    prot_ints = []
    ion_ints = []

    count_prots = 1
    allprots = normed_df.index.get_level_values(0).unique()
    quantified_prots = []

    for protein in allprots:
        if(count_prots%100 ==0):
            print(f"prot {count_prots} of {len(allprots)}")
        count_prots+=1

        protvals = pd.DataFrame(normed_df.loc[protein]).copy()#DataFrame definition to avoid pandas Series objects
        protvals = ProtvalCutter(protvals, maximum_df_length=maximum_df_length).get_dataframe()
        summed_pepint = np.nansum(2**protvals)

        if(protvals.shape[1]<2):
            normed_protvals = protvals
        else:
            normed_protvals = lfqnorm.normalize_ion_profiles(protvals)
        
        ion_ints.append(normed_protvals)
        scaled_vec = get_protein_profile_from_shifted_peptides(normed_protvals, summed_pepint, min_nonan)
        if scaled_vec is not None:
            prot_ints.append(scaled_vec)
            quantified_prots.append(protein)

        
    index_for_protein_df = pd.Index(data=quantified_prots, name="protein")
    protein_df = 2**pd.DataFrame(prot_ints, index = index_for_protein_df, columns = normed_df.columns)
    protein_df = protein_df.replace(np.nan, 0)
    
    ion_df = 2**pd.concat(ion_ints)
    ion_df = ion_df.replace(np.nan, 0)
    return protein_df, ion_df

In [4]:
#export
def get_protein_profile_from_shifted_peptides(normalized_peptide_profile_df, summed_pepints, min_nonan):
    intens_vec = get_list_with_protein_value_for_each_sample(normalized_peptide_profile_df, min_nonan)
    intens_vec = np.array(intens_vec)
    summed_intensity = np.nansum(2**intens_vec)
    if summed_intensity == 0: #this means all elements in intens vec are nans
        return None
    intens_conversion_factor = summed_pepints/summed_intensity
    scaled_vec = intens_vec+np.log2(intens_conversion_factor)
    return scaled_vec

def get_list_with_protein_value_for_each_sample(normalized_peptide_profile_df, min_nonan):
    intens_vec = []
    for sample in normalized_peptide_profile_df.columns:
        reps = normalized_peptide_profile_df.loc[:,sample].to_numpy()
        nonan_elems = sum(~np.isnan(reps))
        if(nonan_elems>=min_nonan):
            intens_vec.append(np.nanmedian(reps))
        else:
            intens_vec.append(np.nan)
    return intens_vec


In [5]:
#export
import pandas as pd

class ProtvalCutter():
    def __init__(self, protvals_df, maximum_df_length = 100):
        self._protvals_df = protvals_df
        self._maximum_df_length = maximum_df_length
        self._dataframe_too_long = None
        self._sorted_idx = None
        self._check_if_df_too_long_and_sort_index_if_so()


    def _check_if_df_too_long_and_sort_index_if_so(self):
        self._dataframe_too_long =len(self._protvals_df.index)>self._maximum_df_length
        if self._dataframe_too_long:
            self._determine_nansorted_df_index()

    def _determine_nansorted_df_index(self):
        idxs = self._protvals_df.index
        self._sorted_idx =  sorted(idxs, key= lambda idx : self._get_num_nas_in_row(self._protvals_df.loc[idx]))
        
    @staticmethod
    def _get_num_nas_in_row(row):
        return sum(np.isnan(row.to_numpy()))


    def get_dataframe(self):
        if self._dataframe_too_long:
            return self._get_shortened_dataframe()
        else:
            return self._protvals_df

    def _get_shortened_dataframe(self):
        shortened_index = self._sorted_idx[:self._maximum_df_length]
        return self._protvals_df.loc[shortened_index]



### Unit Tests

#### Classes for testcase generation

In [6]:
import numpy as np
import pandas as pd

from  numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

class ProteinProfileGenerator():
    def __init__(self, peptide_profiles):
        self._peptide_profiles = peptide_profiles
        
        self.protein_profile_dataframe = None
        self._generate_protein_profile_dataframe()

    def _generate_protein_profile_dataframe(self):
        collected_profiles = [x.peptide_profile_vector for x in self._peptide_profiles]
        protnames_for_index = [x.protein_name for x in self._peptide_profiles]
        pepnames_for_index = [f'{idx}' for idx in range(len(self._peptide_profiles))]
        self.protein_profile_dataframe = pd.DataFrame(collected_profiles,index=[protnames_for_index, pepnames_for_index])
        self.protein_profile_dataframe = np.log2(self.protein_profile_dataframe.replace(0, np.nan))



class PeptideProfile():
    def __init__(self, protein_name, fraction_zeros_in_profile, systematic_peptide_shift, add_noise, num_samples = 20, min_intensity = 1e6, max_intensity = 1e10):


        self._fraction_zeros_in_profile = fraction_zeros_in_profile
        self._systematic_peptide_shift = systematic_peptide_shift
        self._add_noise = add_noise
        self._min_intensity = min_intensity
        self._max_intensity = max_intensity
        self._num_samples = num_samples

        self.protein_name = protein_name
        self.peptide_profile_vector = []
        self._define_peptide_profile_vector()

    def _define_peptide_profile_vector(self):
        self.peptide_profile_vector = self._get_single_peptide_profile_template()
        self._scale_profile_vector()
        if self._add_noise:
            self._apply_poisson_noise_to_profilevector()
        self._add_zeros_to_profilevector()

    def _get_single_peptide_profile_template(self):
        rs = RandomState(MT19937(SeedSequence(42312)))
        return rs.randint(low=self._min_intensity, high=self._max_intensity,size=self._num_samples)

    def _scale_profile_vector(self):
        self.peptide_profile_vector = self.peptide_profile_vector*self._systematic_peptide_shift

    def _apply_poisson_noise_to_profilevector(self):
        self.peptide_profile_vector = np.random.poisson(lam=self.peptide_profile_vector, size=len(self.peptide_profile_vector))

    def _add_zeros_to_profilevector(self):
        num_elements_to_set_zero = int(self._num_samples*self._fraction_zeros_in_profile)
        idxs_to_set_zero = np.random.choice(self._num_samples,size=num_elements_to_set_zero, replace=False)
        self.peptide_profile_vector[idxs_to_set_zero] = 0
        


#### Tests

In [7]:
import pandas as pd
import numpy as np
#test df cutting

def test_sorting_by_num_nans():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['P', 'P', 'P'],['A', 'B', 'C']])
    pcutter = ProtvalCutter(df,maximum_df_length=2)
    sorted_idx = pcutter._sorted_idx
    df_sorted = df.loc[sorted_idx]
    display(df)
    display(df_sorted)
    assert np.allclose(df_sorted.iloc[2].to_numpy(), vals1,equal_nan=True)
    assert np.allclose(df_sorted.iloc[0].to_numpy(), vals3,equal_nan=True)
    

def test_cutting_of_df():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C']])
    pcutter = ProtvalCutter(df, maximum_df_length=2)
    cut_df = pcutter.get_dataframe()
    ion_idx = [x[0] for x in cut_df.index]
    print(ion_idx)
    assert ion_idx == ['C', 'B']





test_sorting_by_num_nans()
test_cutting_of_df()

0    1    2   3
P A  9.0  NaN  NaN NaN
  B  5.0  6.0  NaN NaN
  C  1.0  2.0  3.0 NaN

0    1    2   3
P C  1.0  2.0  3.0 NaN
  B  5.0  6.0  NaN NaN
  A  9.0  NaN  NaN NaN

['C', 'B']


In [8]:
def test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=False)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=3, add_noise=False)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=0.1, add_noise=False)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=100, add_noise=False)
    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    normed_ion_profile = lfqnorm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,11].dropna().to_numpy()
    display(column_from_shifted)
    assert np.allclose(column_from_shifted, column_from_shifted[0])
    
test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other()



0   1          2          3          4          5          6   \
protA 0  44.602965 NaN  41.489085  43.216307  43.505708  42.338404  43.454211   
      1        NaN NaN        NaN        NaN        NaN        NaN        NaN   
      2  29.730290 NaN  26.616410  28.343632  28.633033  27.465729  28.581537   
      3        NaN NaN        NaN        NaN        NaN        NaN  38.547321   

                7          8          9          10         11         12  \
protA 0  43.102621  41.897534  43.625508  43.261340  40.203349  44.447364   
      1        NaN  31.931750        NaN        NaN        NaN        NaN   
      2  28.229946  27.024859  28.752833  28.388665  25.330674  29.574689   
      3        NaN        NaN        NaN  38.354449        NaN        NaN   

                13         14         15         16         17         18  \
protA 0        NaN  44.692343  44.701690  43.106703  44.630975  43.380650   
      1  34.588243        NaN        NaN        NaN        NaN        NaN   
      2  29.681352        NaN  29.829015  28.234029  29.758301  28.507975   
      3        NaN        NaN        NaN        NaN        NaN        NaN   

                19  
protA 0  43.407413  
      1        NaN  
      2  28.534739  
      3        NaN

0   1          2          3          4          5          6   \
protA 0  44.602965 NaN  41.489085  43.216307  43.505708  42.338404  43.454211   
      1        NaN NaN        NaN        NaN        NaN        NaN        NaN   
      2  44.602965 NaN  41.489085  43.216307  43.505708  42.338404  43.454211   
      3        NaN NaN        NaN        NaN        NaN        NaN  43.454211   

                7          8          9         10         11         12  \
protA 0  43.102621  41.897534  43.625508  43.26134  40.203349  44.447364   
      1        NaN  41.897534        NaN       NaN        NaN        NaN   
      2  43.102621  41.897534  43.625508  43.26134  40.203349  44.447364   
      3        NaN        NaN        NaN  43.26134        NaN        NaN   

                13         14        15         16         17        18  \
protA 0        NaN  44.692343  44.70169  43.106703  44.630975  43.38065   
      1  44.554027        NaN       NaN        NaN        NaN       NaN   
      2  44.554027        NaN  44.70169  43.106703  44.630975  43.38065   
      3        NaN        NaN       NaN        NaN        NaN       NaN   

                19  
protA 0  43.407413  
      1        NaN  
      2  43.407413  
      3        NaN

array([40.20334853, 40.20334853])

In [9]:
def test_that_profiles_with_noise_are_close():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    
    normed_ion_profile = lfqnorm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,9].dropna().to_numpy()

    assert np.allclose(column_from_shifted, column_from_shifted[0],rtol=0.01, atol=0.01)


test_that_profiles_with_noise_are_close()

0          1          2          3          4          5   \
protA 0  44.602965  44.474240  41.489086  43.216306  43.505707  42.338404   
      1  44.602958  44.474246  41.489087  43.216303  43.505713  42.338430   
      2  44.602966  44.474228  41.489201  43.216414  43.505707  42.338317   
      3  44.602965  44.474241  41.489086  43.216307  43.505705  42.338408   

                6          7          8          9          10         11  \
protA 0  43.454211  43.102621  41.897534  43.625508  43.261339  40.203349   
      1  43.454204  43.102600  41.897532  43.625486  43.261359  40.203348   
      2  43.454176  43.102554  41.897503  43.625583  43.261402  40.203281   
      3  43.454210  43.102624  41.897530  43.625506  43.261340  40.203348   

                12         13         14         15         16         17  \
protA 0  44.447364  44.554027  44.692343  44.701690  43.106703  44.630976   
      1  44.447367  44.554012  44.692351  44.701691  43.106709  44.630971   
      2  44.447370  44.553960  44.692316  44.701766  43.106707  44.630907   
      3  44.447364  44.554030  44.692342  44.701692  43.106704  44.630978   

                18         19  
protA 0  43.380650  43.407414  
      1  43.380629  43.407425  
      2  43.380703  43.407505  
      3  43.380649  43.407408

In [10]:
def test_that_protein_intensities_are_retained():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    
    peptide_profiles = [peptide1, peptide2, peptide3, peptide4]
    summed_intensity_protein = sum([np.nansum(x.peptide_profile_vector) for x in peptide_profiles])
    
    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    
    protein_df_normed, _ = estimate_protein_intensities(protein_df, min_nonan=1, maximum_df_length=100)
    display(protein_df_normed)
    summed_lfq_intensities = np.sum(protein_df_normed.iloc[0].to_numpy())
    assert np.allclose(summed_lfq_intensities, summed_intensity_protein)

test_that_protein_intensities_are_retained()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
protein,,,,,,,,,,,,,,,,,,,,
protA,2.430032e+13,2.222604e+13,2.806987e+12,9.293653e+12,1.135807e+13,5.057221e+12,1.095972e+13,8.589422e+12,3.725588e+12,1.234150e+13,9.588315e+12,1.151312e+12,2.181582e+13,2.348985e+13,2.585351e+13,2.602149e+13,8.613754e+12,2.477672e+13,1.041502e+13,1.060992e+13


In [11]:
def run_with_multiple_proteins():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protB",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide5= PeptideProfile(protein_name="protC",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide6= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide7= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide8= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    peptide_profiles = [peptide1, peptide2, peptide3, peptide4, peptide5, peptide6, peptide7, peptide8]
    protein_df = ProteinProfileGenerator(peptide_profiles).protein_profile_dataframe
    protein_df_normed, _ = estimate_protein_intensities(protein_df, min_nonan=1, maximum_df_length=100)
    display(protein_df_normed)
    
run_with_multiple_proteins()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
protein,,,,,,,,,,,,,,,,,,,,
protA,2.404741e+13,2.199484e+13,2.777891e+12,9.196948e+12,1.124009e+13,5.004590e+12,1.084576e+13,8.500037e+12,3.686832e+12,1.221308e+13,9.488625e+12,1.139325e+12,2.158881e+13,2.324543e+13,2.558420e+13,2.575068e+13,8.524119e+12,2.451851e+13,1.030660e+13,1.049959e+13
protB,8.906546e+11,8.146290e+11,1.028816e+11,3.406306e+11,4.162960e+11,1.853566e+11,4.016983e+11,3.148182e+11,1.365502e+11,4.523401e+11,3.514310e+11,4.219816e+10,7.995916e+11,8.609494e+11,9.475779e+11,9.537368e+11,3.157103e+11,9.081155e+11,3.817296e+11,3.888771e+11
protC,8.906531e+11,8.146253e+11,1.028815e+11,3.406305e+11,4.162939e+11,1.853572e+11,4.016988e+11,3.148183e+11,1.365501e+11,4.523409e+11,3.514310e+11,4.219817e+10,7.995925e+11,8.609502e+11,9.475781e+11,9.537366e+11,3.157110e+11,9.081155e+11,3.817288e+11,3.888773e+11
protD,2.671960e+12,2.443885e+12,3.086449e+11,1.021891e+12,1.248887e+12,5.560707e+11,1.205095e+12,9.444563e+11,4.096502e+11,1.357020e+12,1.054293e+12,1.265945e+11,2.398777e+12,2.582848e+12,2.842731e+12,2.861207e+12,9.471346e+11,2.724345e+12,1.145190e+12,1.166632e+12


## Learning tests

In [12]:
import pandas as pd
import numpy as np

def test_that_dataframe_is_generated_as_expected():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=['A', 'A', 'A'])
    display(df)
    assert df.iloc[2, 2] == 11
    assert df.iloc[1, 2] == 7



test_that_dataframe_is_generated_as_expected()

,0,1,2,3
A,1,2,3,4
A,5,6,7,8
A,9,10,11,12


In [13]:
def test_retrieval_of_numpy_arrays_from_dataframe():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C'], ['a', 'b', 'a']])
    display(df)
    assert np.allclose(vals2, df.loc['B'])
    assert np.allclose([2, 6, 10], df.loc[:,1])

test_retrieval_of_numpy_arrays_from_dataframe()

,,0,1,2,3
A,a,1,2,3,4
B,b,5,6,7,8
C,a,9,10,11,12


In [14]:
def test_setting_numpy_seed():
    from numpy.random import MT19937
    from numpy.random import RandomState, SeedSequence

    rs = RandomState(MT19937(SeedSequence(42)))
    res = rs.randint(10,size=20)
    display(res)

test_setting_numpy_seed()

array([2, 6, 8, 8, 3, 3, 3, 3, 4, 7, 2, 7, 5, 4, 0, 8, 1, 3, 7, 1])